# Descrição do Problema

O conjunto de dados em questão foi disponibilizado pelo Banco Santander em uma competição no site Kaggle. O problema consiste em identificar a partir das variáveis quais clientes estão insatisfeitos e são propensos a deixar o Banco. O conjunto de dados é completamente anonimizado.

Link para a competição no Kaggle: __[Santander Customer Satisfactions](https://www.kaggle.com/c/santander-customer-satisfaction)__

## 1. Primeiros Passos

Inicialmente, importaremos as bibliotecas necessárias e carregaremos os dados na memória.

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv("data/train.csv")
train_df.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [12]:
train_df.TARGET.astype('bool').value_counts()

TARGET
False    73012
True      3008
Name: count, dtype: int64

In [11]:
from collections import Counter
Counter(train_df["TARGET"])

Counter({0: 73012, 1: 3008})